In [20]:
## import libraries

import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import time
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import re
from lightgbm import early_stopping
from lightgbm import log_evaluation
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
#from google.cloud import storage
import os
import json
import plotly.express as px

import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("o9_logger")

pd.options.display.max_rows = 25
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 100
pd.set_option("display.width", 1000)
pd.options.display.precision = 3

In [2]:
## functions

def calculate_mape(target, predicted):
    """
    Calculate the Mean Absolute Percentage Error (MAPE) between ytest and ypred.
    
    Parameters
    ----------
    ytest : array-like
        The true values.
    ypred : array-like
        The predicted values.
    
    Returns
    -------
    float: The MAPE between ytest and ypred.
    """
    # Convert inputs to NumPy arrays
    ytest = np.array(target)
    ypred = np.array(predicted)
    
    # Calculate the absolute percentage error for each point
    abs_pct_error = np.abs((target - predicted) / target)
    
    # Handle divide by zero error
    abs_pct_error[np.isinf(abs_pct_error)] = 0.0
    
    # Calculate the mean absolute percentage error
    mape = np.mean(abs_pct_error) * 100
    
    # Calculate the weighted mean absolute percentage error
    wmape = np.sum(np.abs(target - predicted))/np.sum(target)
    
    return mape, wmape

def plot_regression_accuracy(target, predicted, max_error=100, max_range=100):
    # Calculate the R-squared and MSE, mape metrics
    r2 = r2_score(target, predicted)
    rmse_accuracy = np.sqrt(np.mean((target - predicted)**2))
    mape,wmape = calculate_mape(target, predicted)
    
    # Create a scatter plot of the target and predicted values
    plt.figure(figsize=(12, 8), dpi=200)
    errors = [abs(t - p) for t, p in zip(target, predicted)]
    filtered_target = []
    filtered_predicted = []
    for t, p, e in zip(target, predicted, errors):
        if e <= max_error and t <= max_range and p <= max_range:
            filtered_target.append(t)
            filtered_predicted.append(p)
    plt.scatter(filtered_target, filtered_predicted, s=4, alpha=0.3)

    # Add a line indicating perfect predictions
    min_val = min(filtered_target + filtered_predicted)
    max_val = max(filtered_target + filtered_predicted)
    plt.plot([min_val, max_val], [min_val, max_val], 'k--')

    # Set the axis labels and plot title
    plt.xlabel('Target Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Regression Accuracy (R-squared={r2:.2f}, RMSE={rmse_accuracy:.2f},MAPE={mape:.2f}%,WMAPE={wmape:.2f}%)')

    # Show the plot
    plt.show()

    # Create a histogram of the error distribution
    plt.figure(figsize=(8, 6), dpi=200)
    filtered_errors = [t - p for t, p in zip(filtered_target, filtered_predicted)]
    sns.histplot(filtered_errors, kde=True)
    

    # Set the axis labels and plot title
    plt.xlabel('Error')
    plt.ylabel('Frequency')
    plt.title('Error Distribution')

    # Show the plot
    plt.show()
    return

In [ ]:
## Read base data

logger.info("------ 1. Read data ------")

try:
    sales_df = actuals.copy() 
    price_df = price_data.copy()
    itemmaster_df = itemmaster.copy() 
    dimtime = timemaster.copy() 
    assortment_df = assortment_data.copy() 
    current_week = CurrentWeek.copy()
    location_master = locationmaster.copy()
    itemseasonweek = itemseason_week.copy()

except:
    sales_df = pd.read_csv("new_data/actuals.csv")
    price_df = pd.read_csv("new_data/price_data.csv")
    itemmaster_df = pd.read_csv("new_data/itemmaster.csv")
    dimtime = pd.read_csv("data/timemaster.csv")
    assortment_df = pd.read_csv("data/assortment_data.csv")
    current_week = pd.read_csv("data/current_week.csv")
    location_master = pd.read_csv("data/location_master.csv")
    itemseasonweek = pd.read_csv('data/itemseasonweek.csv')

logger.info("sales_df head:{}".format(sales_df.head()))
logger.info("price_df head:{}".format(price_df.head()))
logger.info("itemmaster_df head:{}".format(itemmaster_df.head()))
logger.info("assortment_df head:{}".format(assortment_df.head()))
logger.info("itemseasonweek head:{}".format(itemseasonweek.head()))
logger.info("location_master head:{}".format(location_master.head()))

In [3]:
logger.info("------ 1.5 Define Variables ------")

itemcol = 'Item.[Planning Item]'
loccol = "Location.[Location Country]"
timecol = "Time.[WeekKey]"
actual = "Actual"
item_season = 'EB_Item_Seasonality.[EB_Item_Seasonality]'
is_assorted = 'EB_Assorted'

price_start_date = "Price_Valid_From"
price_end_date = "Price_Valid_To"
regular_price = "EB_Future_Display Price"
final_price = "EB_Future_Final_Price"
mkd_perc = "markdown_perc"

week_rank = 'weekrank'
week_rank_max = 'week_rank_max'
week_rank_mkd = 'week_rank_mkd'
window = "window%"
median_window = 'median_values'
mfp_level = "Item.[EB_Dept_Brand_BM]"

versioncol = 'Version.[Version Name]'

assortment_start_date = "EB_AdjStartWeek"
assortment_end_date = "EB_AdjEndWeek"

markdown_level = [itemcol,loccol]  # the level at which markdown is done and planned

input_version = sales_df[versioncol].iloc[0]

INFO:o9_logger:------ 1.5 Define Variables ------


In [4]:
logger.info("------ 2. Process Input Data ------")

## process time dim
dimtime.dropna(axis=1,inplace=True)

## process itemmaster
itemmaster_df[is_assorted].fillna(False,inplace=True)

## process assortment data
assortment_df['EB_NoMarkDown'].fillna(False,inplace=True)
assortment_df = assortment_df.dropna()  # drop rows where data is incomplete

## get current_week
current_week = current_week[timecol].iloc[0]
current_week = pd.to_datetime(current_week)

## process sales_df
# get proper time column 
sales_df = sales_df.merge(dimtime,on='Time.[Week]',how = 'left')
sales_df = sales_df[[itemcol,loccol,timecol,actual]].reset_index(drop=True)
sales_df[timecol] = pd.to_datetime(sales_df[timecol], format='%m/%d/%Y %I:%M:%S %p')
# filter sales data from 2 years back to 2 weeks back
date_2_years_back = current_week + pd.DateOffset(years=-2)
date_2_weeks_back = current_week + pd.DateOffset(weeks=-2)
sales_df = sales_df[(sales_df[timecol]>= date_2_years_back)&(sales_df[timecol]<= date_2_weeks_back)].reset_index(drop=True)


## process price_df (price missing for 7901 combinations or 0.95 %)
# convert date columns to datetime
price_df[price_start_date] = pd.to_datetime(price_df[price_start_date], format='%m/%d/%Y %I:%M:%S %p')
price_df[price_end_date] = pd.to_datetime(price_df[price_end_date], format='%m/%d/%Y %I:%M:%S %p')
price_df = price_df[[itemcol,loccol,price_start_date,price_end_date,regular_price,final_price]].reset_index(drop=True)
# calculate markdown perc 
price_df[mkd_perc] = (price_df[regular_price] - price_df[final_price])/price_df[regular_price]
# sort the price data
price_df = price_df.sort_values(markdown_level+[price_start_date])
# reduce price data to only those combinations where sales data is present
price_df = price_df.merge(sales_df[markdown_level].drop_duplicates().reset_index(drop=True),on = markdown_level,how='inner')
# step 1 fill na
price_df.fillna(0,inplace=True)
# step 2 consolidate the dates based on markdown
price_df = price_df.groupby(markdown_level+[mkd_perc]).agg({price_start_date:'min',price_end_date:'max',regular_price:'mean',final_price:'mean'}).reset_index()
# step 3 remove overlap if present between markdown dates
def remove_overlapping_dates(group):
    for i in range(1, len(group)):
        # If there is an overlap between dates of 2 consecutive markdowns
        if (group.iloc[i][price_start_date] <= group.iloc[i-1][price_end_date]):  #and (group.iloc[i-1][mkd_perc] > 0)
            # Adjust the 'Price_Valid_To' date of the earlier period
            group.iloc[i-1, group.columns.get_loc(price_end_date)] = group.iloc[i][price_start_date] - pd.Timedelta(days=7)
    return group

price_df = price_df.sort_values(markdown_level+[price_start_date])
price_df = price_df.groupby(markdown_level, group_keys=False).apply(remove_overlapping_dates).reset_index(drop=True)  # 5 min

INFO:o9_logger:------ 2. Process Input Data ------


In [5]:
logger.info("------ 3. Combine Sales and price data ------")

# merge actuals and price data (take zero filled actuals)
merged_df = sales_df.merge(price_df,on=markdown_level,how='left')
# filter rows so that each week has corresponding price (check for nulls)
merged_df = merged_df[(merged_df[timecol] >= merged_df[price_start_date]) & (merged_df[timecol] <= merged_df[price_end_date])].reset_index(drop=True)

INFO:o9_logger:------ 3. Combine Sales and price data ------


In [6]:
logger.info("------ 4. Window Prediction------")
# filter relevant data
merged_df = merged_df[markdown_level +[timecol,actual,mkd_perc]].reset_index(drop=True)
# sort data
merged_df = merged_df.sort_values(markdown_level+[timecol]).reset_index(drop=True)

#assign ranks/numbering to the product lifecycle
merged_df[week_rank] = merged_df.groupby(markdown_level)[timecol].rank()
merged_df[week_rank_max] = merged_df.groupby(markdown_level)[week_rank].transform(max)


def first_positive_promotion_optimized(df):
    # Identify the first positive 'promotion %' for each item-location combination
    first_positive = df[df[mkd_perc] > 0].groupby(markdown_level)[week_rank].transform('min')

    # Assign this value to a new column in the original DataFrame
    # We use reindex to align with the original DataFrame's index
    df[week_rank_mkd] = first_positive.reindex(df.index, fill_value=0)
    
    df[week_rank_mkd] = df.groupby(markdown_level)[week_rank_mkd].transform('max')

    return df

# Identify the week in the lifecycle when markdown started
merged_df = first_positive_promotion_optimized(merged_df)

# calculate window start %
merged_df[window] = merged_df[week_rank_mkd]/merged_df[week_rank_max]

# merge MFP information for each planning item (check distribution data here)
merged_df = merged_df.merge(itemmaster_df[[itemcol,mfp_level]].drop_duplicates(),on=itemcol,how='inner')

# filter only positive markdowns and then calculate median window at MFP markdown_level
mkd_window_df = merged_df[merged_df[window]>0][markdown_level+[mfp_level,window]].drop_duplicates().groupby(mfp_level)[window].agg(median_values='median', count_values='count').reset_index()

# store the window values so that they can be used in real time forecasting
save_metadata = {}
save_metadata['df_w'] = mkd_window_df.to_dict()

INFO:o9_logger:------ 4. Window Prediction------
/var/folders/hm/ktk0fh81691ccjf3l0h99tq80000gn/T/ipykernel_67432/1457374490.py:9: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  merged_df[week_rank_max] = merged_df.groupby(markdown_level)[week_rank].transform(max)


In [8]:
# filter out those values which are equal to 1 as they are basically no markdown on the entire lifecycle
# mkd_window_df = mkd_window_df[mkd_window_df['median_values']<1].reset_index(drop=True)
# make the markdown as zero percentage as well

In [7]:
logger.info("------ 5. Depth prediction starts------")
logger.info("--- 5.1 data prep ---")

adj_mkd = 'adj_mkd %'

# Filter the DataFrame to include only rows with mkd_perc > 0
mkd_sales = merged_df[merged_df[mkd_perc] > 0]

# Function to calculate weighted average for each group
def weighted_avg(group):
    if group[actual].sum() == 0:
        return group[mkd_perc].mean()   # don't return zero if no sales occured
    else:
        return (group[actual] * group[mkd_perc]).sum() / group[actual].sum()

# Calculate the weighted average promotion % for each item
weighted_avg_mkd = mkd_sales.groupby(markdown_level).apply(weighted_avg).reset_index(name=adj_mkd)

# Merge the new promotion % back into the original DataFrame
merged_df = merged_df.merge(weighted_avg_mkd, on=markdown_level, how='left')

# Fill NaN values in the new promotion % column with 0 (for items with no markdown)
merged_df[adj_mkd].fillna(0, inplace=True)

# create dataset for depth prediction
mkd_depth_df = merged_df[markdown_level+[adj_mkd]].drop_duplicates().reset_index(drop=True)

# merge item master
mkd_depth_df = mkd_depth_df.merge(itemmaster_df, on = itemcol, how='left')

# perform some cleaning
mkd_depth_df.replace("UNDEFINED", np.nan, inplace=True)
mkd_depth_df.dropna(axis=1, how='all', inplace=True)

INFO:o9_logger:------ 5. Depth prediction starts------
INFO:o9_logger:--- 5.1 data prep ---


In [10]:
logger.info("--- 5.2 train and test data ---")

# Split into training and test sets
X = mkd_depth_df.drop(columns=[adj_mkd,itemcol,versioncol,is_assorted, actual]) 
y = mkd_depth_df[adj_mkd]

save_metadata['train_columns']=X.columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=10)

X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))


cat_features=X_train.dtypes[X_train.dtypes == 'object'].index.to_list()
for c in cat_features:
    X_train[c] = X_train[c].astype('category')
    X_test[c] = X_test[c].astype('category')

save_metadata['cat_columns'] = cat_features

# Create LightGBM dataset
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature = cat_features)
lgb_test = lgb.Dataset(X_test, y_test, categorical_feature = cat_features)

INFO:o9_logger:--- 5.2 train and test data ---


In [60]:
logger.info("--- 5.3 ML training ---")

# Set LightGBM hyperparameters
params = {
    'objective': 'tweedie',  # regression gamma mape poisson tweedie   (mape,mape)
    'metric': '',  # rmse mape
    'min_data_in_leaf':25,
    'learning_rate': 0.02, #0.05 0.02
    'num_iterations' : 4000, 
    'num_leaves' : 50,
    "max_depth" : 8,
    "n_jobs" : 5,
}

# 'tweedie_variance_power': 1.5

# Train LightGBM model
model = lgb.train(params, lgb_train,categorical_feature = cat_features,valid_sets=[lgb_test],callbacks=[early_stopping(50), log_evaluation(50)])

# %%
# Get predictions
y_pred = model.predict(X_test)
y_compare_lgbm = y_test.to_frame().copy()
y_compare_lgbm['pred'] = y_pred

y_pred[y_pred < 0.1] = 0


# Get accuracy on test set
rmse_accuracy = np.sqrt(np.mean((y_test - y_pred)**2))
logger.info('Test RMSE: {}'.format(rmse_accuracy))

#lgb.plot_importance(model,height = 0.5,dpi=200,figsize=(15,15),importance_type='gain')
#plot_regression_accuracy(y_test,y_pred,1,1)

# Calculate feature importance
#feature_importance = model.feature_importance(importance_type='gain')

# Normalize the feature importance scores
#normalized_importance = feature_importance / np.sum(feature_importance)

#MLimportance = pd.DataFrame({"Features":model.feature_name(),"Importance":normalized_importance})

/Users/yashvardhansingh/miniconda3/envs/yashml/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003666 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1041
[LightGBM] [Info] Number of data points in the train set: 144749, number of used features: 43
[LightGBM] [Info] Start training from score -1.436190
Training until validation scores don't improve for 50 rounds
[50]	valid_0's tweedie: 1.92069
[100]	valid_0's tweedie: 1.91037
[150]	valid_0's tweedie: 1.90455
[200]	valid_0's tweedie: 1.90044
[250]	valid_0's tweedie: 1.89727
[300]	valid_0's tweedie: 1.89466
[350]	valid_0's tweedie: 1.89248
[400]	valid_0's tweedie: 1.8907
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[450]	valid_0's tweedie: 1.88898
[500]	valid_0's tweedie: 1.88753
[LightGBM] [Warning] No further splits with positive 

In [61]:
logger.info("--- 5.4 Get Prediction data from assortment ---")

mkd_output_level = [itemcol,'Location.[Location]']

## process and filter assortment data
assortment_df[assortment_start_date] = pd.to_datetime(assortment_df[assortment_start_date], format='%m/%d/%Y %I:%M:%S %p')
assortment_df[assortment_end_date] = pd.to_datetime(assortment_df[assortment_end_date], format='%m/%d/%Y %I:%M:%S %p')

## filter the data
date_3_years_later = current_week + pd.DateOffset(years=3)
date_x_weeks_back = current_week + pd.DateOffset(weeks=-26)

## filter assortment_df for Pre-season  and inseason mkdn prediction
date_3_years_later = current_week + pd.DateOffset(years=3)
date_x_weeks_back = current_week + pd.DateOffset(weeks=-26)

assortment_df_preseason = assortment_df.loc[(assortment_df['EB_Assorted']==True)&(assortment_df['EB_NoMarkDown']==False)&
                                       (assortment_df[assortment_end_date] < date_3_years_later)&
                                       (assortment_df[assortment_start_date] > date_x_weeks_back),:].reset_index(drop=True)

## filter assortment_df for old stock markdown generation
date_x_weeks_back = current_week + pd.DateOffset(weeks=-80)  # go back around 1.5 years

assortment_df_old_stock = assortment_df.loc[(assortment_df['EB_Assorted']==False)&(assortment_df['EB_NoMarkDown']==False)&
                                       (assortment_df[assortment_end_date] < current_week)&
                                       (assortment_df[assortment_start_date] > date_x_weeks_back),:].reset_index(drop=True)


In [62]:
logger.info("--- 5.5 Preseason markdown Prediction ---")

## get location info - country
assortment_df_preseason = assortment_df_preseason.merge(location_master[['Location.[Location]', 'Location.[Location Country]']],on='Location.[Location]',how='left')
# get itemmaster info
assortment_df_preseason = assortment_df_preseason.merge(itemmaster_df.drop(['Version.[Version Name]','EB_Assorted', 'Actual'],axis=1), on = itemcol, how='left')
# drop unnecessary columns
assortment_df_preseason = assortment_df_preseason.drop(['Version.[Version Name]','EB_Season.[EB_Season]', 'EB_Item_Seasonality.[EB_Item_Seasonality]'],axis=1)

# remove duplicates created in the assortment data due to season dimension
assortment_df_preseason = assortment_df_preseason.drop_duplicates(subset = mkd_output_level+ [assortment_start_date]).reset_index(drop=True)

# perform some cleaning
assortment_df_preseason.replace("UNDEFINED", np.nan, inplace=True)

## perform mkd_perc % prediction
dfassort_predict = assortment_df_preseason.copy().rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
common_columns = dfassort_predict.columns.intersection(X_train.columns)
for c in cat_features:
    dfassort_predict[c] = dfassort_predict[c].astype('category')
assortment_df_preseason[adj_mkd] = model.predict(dfassort_predict[common_columns])

# adjust the mkd_perc % values
assortment_df_preseason.loc[assortment_df_preseason[adj_mkd]<0.1,adj_mkd] = 0
assortment_df_preseason[adj_mkd] = (assortment_df_preseason[adj_mkd].round(2) / 0.05).round() * 0.05

# get the window values and fill zero where no present
assortment_df_preseason = assortment_df_preseason.merge(mkd_window_df[[mfp_level,median_window]],on = mfp_level,how='left')
assortment_df_preseason[median_window].fillna(0,inplace=True)

## filter the combinations for which output will be given and relevent columns
preseason_mkd_output = assortment_df_preseason[(assortment_df_preseason[median_window]>0) & 
                                               (assortment_df_preseason[adj_mkd]>0)][mkd_output_level+
                                                                                     [assortment_start_date,assortment_end_date,adj_mkd,
                                                                                      median_window]].reset_index(drop=True)

# adjust the start and end dates to the start of the week
preseason_mkd_output[assortment_start_date] = preseason_mkd_output[assortment_start_date] - pd.to_timedelta(preseason_mkd_output[assortment_start_date].dt.weekday, unit='d')
preseason_mkd_output[assortment_end_date] = preseason_mkd_output[assortment_end_date] - pd.to_timedelta(preseason_mkd_output[assortment_end_date].dt.weekday, unit='d')

# Calculate the start_date based on the window percentage
preseason_mkd_output['new_start_date'] = preseason_mkd_output[assortment_end_date] - pd.to_timedelta((1-preseason_mkd_output[median_window]) * (preseason_mkd_output[assortment_end_date] - preseason_mkd_output[assortment_start_date]).dt.days, unit='D')
preseason_mkd_output['new_start_date'] = preseason_mkd_output['new_start_date'] - pd.to_timedelta(preseason_mkd_output['new_start_date'].dt.weekday, unit='d')
preseason_mkd_output['new_start_date'] = preseason_mkd_output['new_start_date'].dt.floor('D')

# explode the dates between the new start and end dates
date_ranges = (
    preseason_mkd_output.apply(lambda row: pd.date_range(start=row['new_start_date'], end=row[assortment_end_date], freq='W-MON'), axis=1)
    .explode()
)

#merge them with the original dataframe and rename the columns
preseason_mkd_output = pd.merge(preseason_mkd_output[mkd_output_level+[adj_mkd]], date_ranges.to_frame(), left_index=True, right_index=True)
preseason_mkd_output = preseason_mkd_output.rename(columns={0:timecol,adj_mkd:'EB_AvgMarkdownPriceLB'}).reset_index(drop=True)

#get markdown values that are greater than the current date
preseason_mkd_output = preseason_mkd_output[preseason_mkd_output[timecol]>current_week].reset_index(drop=True)

# get the tenant format of dates from timemaster
dimtime[timecol] = pd.to_datetime(dimtime[timecol])
preseason_mkd_output = preseason_mkd_output.merge(dimtime,on=timecol,how='left').drop(timecol,axis=1)

# add version col
preseason_mkd_output[versioncol] = input_version

# add season infromation to the data provided 
season_merge_level = [itemcol,'Time.[Week]']
preseason_mkd_output = preseason_mkd_output.merge(itemseasonweek[season_merge_level+[item_season]].drop_duplicates(),on=season_merge_level,how='left')

# drop rows whereever season information is not present
preseason_mkd_output = preseason_mkd_output.dropna()

#filter relevant rows for output
preseason_mkd_output = preseason_mkd_output[[versioncol] + mkd_output_level + ['Time.[Week]',item_season,'EB_AvgMarkdownPriceLB']]

logger.info("preseason_mkd_output head:{}".format(preseason_mkd_output.head()))

In [71]:
logger.info("--- 5.6 Old Stock markdown Prediction ---")

old_stock_mkdn_output = merged_df[(merged_df['weekrank']==merged_df['week_rank_max'])&
                                  (merged_df['markdown_perc']>0)][markdown_level+ [timecol,mkd_perc]].reset_index(drop=True)

## get location info - country for assortment old stock
assortment_df_old_stock = assortment_df_old_stock.merge(location_master[['Location.[Location]', 'Location.[Location Country]']],on='Location.[Location]',how='left')

old_stock_mkdn_output = old_stock_mkdn_output.merge(assortment_df_old_stock[mkd_output_level + [loccol]].drop_duplicates(),on=markdown_level,how='inner')

# add tenant format week inforation
old_stock_mkdn_output = old_stock_mkdn_output.merge(dimtime,on=timecol,how='left').drop(timecol,axis=1)

# add version col
old_stock_mkdn_output[versioncol] = input_version

# add season infromation to the data provided 
season_merge_level = [itemcol,'Time.[Week]']
old_stock_mkdn_output = old_stock_mkdn_output.merge(itemseasonweek[season_merge_level+[item_season]].drop_duplicates(),on=season_merge_level,how='left')

# drop rows whereever season information is not present
old_stock_mkdn_output = old_stock_mkdn_output.dropna()


#filter relevant rows for output
old_stock_mkdn_output = old_stock_mkdn_output[[versioncol] + mkd_output_level + 
                                              [item_season,mkd_perc]].drop_duplicates().reset_index(drop=True).rename(columns={mkd_perc:'EB_MFP_OldStock_Markdown_Percent'})

# add O- to the season column to  show they belong to an older season
old_stock_mkdn_output[item_season] = 'O-' + old_stock_mkdn_output[item_season]

logger.info("old_stock_mkdn_output head:{}".format(old_stock_mkdn_output.head()))

In [ ]:
## Save model to cloud 
logger.info("----- 6.1 save model to cloud  -----")
bucket_name = 'stgte00012file-eu'

## TODO: Change the model file name
model_name = "Pre_season_markdown_lgbm_model.txt"
model.save_model(model_name)     # to load model bst = lgb.Booster(model_file='model.txt')
bucket = storage.Client().bucket(bucket_name)
## TODO: Change the GCS folder name for model in GCS
model_path = "Pre_season_markdown_models/"+model_name
blob = bucket.blob(model_path)
blob.upload_from_filename(model_name)
logger.info("Url for saved blobs: {}".format(blob))
os.remove(model_name)

logger.info("----- 6.2 save metadata to cloud  -----")
      
## TODO: Change the metadata file name
file_name = 'Pre_season_markdown_metadata.json'
with open(file_name, 'w') as outfile:
    json.dump(save_metadata, outfile)
bucket = storage.Client().bucket(bucket_name)
model_path = "Pre_season_markdown_models/"+file_name
blob = bucket.blob(model_path)
blob.upload_from_filename(file_name)
logger.info("Url for saved blobs: {}".format(blob))
os.remove(file_name) 